# **Geração das Matrizes de Adjacência** 

Segunda parte do processo, que consiste em importar os arquivos `.csv` oriundos do QGIS e gerar a matriz de adjacências $A \in \mathbb{R}^{|V| \times |V|}$ assim como o conjunto de vértices $V$ contendo os nomes e os geo-códigos dos 5570 municípios brasileiros.

Como optou-se pela análise de cada modal separadamente, haverá distinção entre as relações de cada meios de transporte, sendo eles *Ferroviário*, *Hidroviário*, *Rodoviário* e *Rotas Aéreas*. Além disso, será gerada uma quinta matriz, que considera como relação de conectividade a existência de froteiras físicas (*Vizinhança*) ao invés de rotas de acesso.

\begin{equation}
    A_{fer}, A_{hid}, A_{rod}, A_{aer}, A_{viz} = \left[a_{ij}\right] \in \mathbb{R}^{|V| \times |V|} \\
\end{equation}

In [1]:
import numpy as np
import pandas as pd
import itertools

In [2]:
# leitura dos dados como um dataframe
PATH = 'dados/gerado/'

# remove tuplas de 1 item e gera arranjos a partir de tuplas de 3+ itens
def rearrange_relations(df):
    
    drop_list = []
    comb_list = []
    
    for idx, data in df.iterrows():
        r = data['relacao']
        
        if len(r) != 2:
            drop_list.append(idx)
            if len(r) > 2:
                comb_list += list(itertools.combinations(r, 2))
    
    df = df.drop(index=drop_list)
    df = df.append(pd.DataFrame({'relacao': comb_list}), ignore_index=True)
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    return df

In [3]:
#-----------------------------------------------------------------------------------------------
#  (1) conexões por rotas aéreas
#-----------------------------------------------------------------------------------------------
df_a = pd.read_csv(PATH + 'malha_aer.csv', low_memory=False)
df_a = df_a.drop(columns=['OBJECTID', 'ORIGEM___L', 'DESTINO___', 'CARGA__KG_'])
df_a = pd.DataFrame(data={'relacao':[ tuple(set(i)) for i in df_a.values ]}).drop_duplicates()
df_a = rearrange_relations(df_a)
df_a

,relacao
0,"(1100304, 1100122)"
1,"(1301704, 1100122)"
2,"(1100122, 1302405)"
3,"(3106200, 1100122)"
4,"(1100122, 3549805)"
...,...
871,"(5103353, 5300108)"
872,"(5103403, 5300108)"
873,"(5107859, 5300108)"
874,"(5208707, 5300108)"


In [4]:
#-----------------------------------------------------------------------------------------------
#  (2) conexões por rotas ferroviárias
#-----------------------------------------------------------------------------------------------
df_f = pd.read_csv(PATH + 'malha_fer.csv', low_memory=False)
df_f = df_f.drop(columns=['OBJECTID', 'FERROVIA', 'OPERADORA', 'CODPNVSIMP', 'COD_PNV', 'nome', 
                          'geometriaa', 'anoderefer'])
df_f = df_f.groupby(by=['xcoord', 'ycoord'])['geocodigo'].apply(set).apply(tuple)
df_f = pd.DataFrame(data={'relacao':df_f.values}).drop_duplicates()
df_f = rearrange_relations(df_f)
df_f

,relacao
0,"(5005202, 5003207)"
1,"(5005608, 5003207)"
2,"(4322400, 4300406)"
3,"(5005608, 5001102)"
4,"(1505304, 1505106)"
...,...
1267,"(2607901, 2602902)"
1268,"(2513703, 2501807)"
1269,"(2507507, 2501807)"
1270,"(2609600, 2611606)"


In [5]:
#-----------------------------------------------------------------------------------------------
#  (3) conexões por rotas hidroviárias
#-----------------------------------------------------------------------------------------------
df_h = pd.read_csv(PATH + 'malha_hid.csv', low_memory=False)
df_h = df_h.drop(columns=['geometriaa', 'geometri_1', 'operaciona', 'situacaofi', 'regime', 
                          'extensaotr', 'tipoembarc', 'path', 'caladomaxs', 'OBJECTID_1', 
                          'nome_2', 'tipouso', 'anoderefer'])

temp = df_h[ (df_h['layer']=='tra_travessia_l') & df_h['nome'].notna() & (df_h['nome'] != '<Null>')]
temp = temp.groupby('nome')['geocodigo'].apply(set).apply(tuple)
temp = pd.DataFrame(data={'relacao': temp.values}).drop_duplicates()

df_h = df_h.groupby(['xcoord', 'ycoord'])['geocodigo'].apply(set).apply(tuple)
df_h = pd.DataFrame(data={'relacao':df_h.values}).drop_duplicates()
df_h = df_h.append(temp, ignore_index=True).drop_duplicates()
df_h = rearrange_relations(df_h)
df_h

,relacao
0,"(1200203, 1200427)"
1,"(1304062, 1300607)"
2,"(1303908, 1300607)"
3,"(1303908, 1304062)"
4,"(1303908, 1300060)"
...,...
1235,"(2706802, 2706703)"
1236,"(2800704, 2706802)"
1237,"(2703601, 2707404)"
1238,"(3550704, 3520400)"


In [6]:
#-----------------------------------------------------------------------------------------------
#  (4) conexões por rotas rodoviárias
#-----------------------------------------------------------------------------------------------
df_r = pd.read_csv(PATH + 'malha_rod.csv', low_memory=False)
df_r = df_r.drop(columns=['geometriaa', 'tipovia', 'jurisdicao', 'administra', 'concession',
                          'revestimen', 'operaciona', 'situacaofi', 'canteirodi', 'nrpistas', 
                          'nrfaixas', 'trafego', 'tipopavime', 'OBJECTID', 'DescSeg', 'TipoPNV', 
                          'CODIGO', 'layer', 'path', 'nome', 'geometri_1', 'anoderefer'])
df_r = df_r.groupby(by=['xcoord', 'ycoord'])['geocodigo'].apply(set).apply(tuple)
df_r = pd.DataFrame(data={'relacao':df_r.values}).drop_duplicates()
df_r = rearrange_relations(df_r)
df_r

,relacao
0,"(1200336, 1200427)"
1,"(1200336, 1301654)"
2,"(1200336, 1200203)"
3,"(1300201, 1301654)"
4,"(1200203, 1200427)"
...,...
12181,"(2500601, 2511905)"
12182,"(2503001, 2511905)"
12183,"(2503209, 2507507)"
12184,"(2511905, 2504603)"


In [7]:
#-----------------------------------------------------------------------------------------------
#  (5) conexões por compartilhamento de fronteira terrestre
#-----------------------------------------------------------------------------------------------
df_v = pd.read_csv(PATH + 'malha_fro.csv', low_memory=False)
df_v = df_v.drop(columns=['geometriaa', 'nome', 'anoderefer', 'geometri_1', 'nome_2', 'anoderef_1'])
df_v = pd.DataFrame(data={'relacao':[ tuple(set(i)) for i in df_v.values ]}).drop_duplicates()
df_v = rearrange_relations(df_v)
df_v

,relacao
0,"(2900306, 2901908)"
1,"(2900306, 2927002)"
2,"(2900306, 2910602)"
3,"(2900306, 2909604)"
4,"(2913705, 2901908)"
...,...
18503,"(3203809, 3203403)"
18504,"(3203809, 3201209)"
18505,"(3202553, 3202009)"
18506,"(3202553, 3200201)"


In [8]:
# checagem de conectividade (adjacência) dos municípios
conn = pd.read_csv(PATH + 'lista_mun.csv')
conn = conn.drop(columns=['geometriaa'])
conn = conn.set_index('geocodigo')
conn['adj'] = np.zeros(len(conn), dtype=bool)

def check_conn(df_arr, df_con):
    
    for df in df_arr:
        for r in df['relacao'].values:
            if len(r) == 2:
                df_con.loc[r[0], 'adj'] = True
                df_con.loc[r[1], 'adj'] = True

    cont = df_con[ ~df_con['adj'] ].count().values[0]
    perc = (cont / len(df_con)) * 100
    
    return str (f"{cont} municípios não foram atingidos ({perc:.2f}% desconexo)")

# contagem por malha viária
print('[AER]', check_conn([df_a], conn.copy()))
print('[FER]', check_conn([df_f], conn.copy()))
print('[HID]', check_conn([df_h], conn.copy()))
print('[ROD]', check_conn([df_r], conn.copy()))
print('[FRO]', check_conn([df_v], conn.copy()))

# contagem total (somente das malhas viárias)
print('\nAo todo,', check_conn([df_a, df_f, df_h, df_r], conn))
print('por rotas de transporte, dentre os', len(conn), 'municípios brasileiros.')

conn[ conn['adj']==False ]

[AER] 5421 municípios não foram atingidos (97.32% desconexo)
[FER] 4361 municípios não foram atingidos (78.29% desconexo)
[HID] 4645 municípios não foram atingidos (83.39% desconexo)
[ROD] 16 municípios não foram atingidos (0.29% desconexo)
[FRO] 5 municípios não foram atingidos (0.09% desconexo)

Ao todo, 2 municípios não foram atingidos (0.04% desconexo)
por rotas de transporte, dentre os 5570 municípios brasileiros.


,nome,xcoord,ycoord,adj
geocodigo,,,,
1301951,Itamarati,-68.252987,-6.428991,False
1200328,Jordão,-71.948674,-9.193355,False


In [9]:
# junção de todas as malhas de conectividade
rel = pd.DataFrame(columns=['relacao', 'geocodigo_1', 'geocodigo_2', 'distancia'])
for df, tag in zip([df_a, df_f, df_h, df_r, df_v], 
                   ['AER', 'FER', 'HID', 'ROD', 'FRO']):
    df[tag] = True
    rel = rel.merge(df, on=['relacao'], how='outer')

# preenchimento da coluna de distancias
dist = []
for idx, r in enumerate(rel['relacao'].values):
    p0, p1 = conn.loc[ r[0] ], conn.loc[ r[1] ]
    dx = (p0['xcoord'] - p1['xcoord'])**2
    dy = (p0['ycoord'] - p1['ycoord'])**2
    dist.append(np.sqrt(dx + dy))

rel['distancia'] = dist # medida em deg²

# transformação da tupla em duas colunas e preenchimento das colunas de máscara
rel['geocodigo_1'] = [ r[0] for r in rel['relacao'].values ]
rel['geocodigo_2'] = [ r[1] for r in rel['relacao'].values ]
rel.drop(columns=['relacao'], inplace=True)
rel = rel.fillna(False)

rel.to_csv(PATH + 'lista_rel.csv', index=False)
rel

,geocodigo_1,geocodigo_2,distancia,AER,FER,HID,ROD,FRO
0,1100304,1100122,2.592404,True,False,False,False,False
1,1301704,1100122,3.542156,True,False,False,False,False
2,1100122,1302405,4.609503,True,False,False,False,False
3,3106200,1100122,20.123000,True,False,False,False,False
4,1100122,3549805,16.020910,True,False,False,False,False
...,...,...,...,...,...,...,...,...
19364,3304755,3203403,0.485587,False,False,False,False,True
19365,3203809,3201209,0.238381,False,False,False,False,True
19366,3202553,3202009,0.241375,False,False,False,False,True
19367,3202553,3200201,0.259158,False,False,False,False,True


# **Exibição dos dados**

In [13]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

rel = pd.read_csv(PATH + 'lista_rel.csv', low_memory=False)
IMG = 'imagens/'

In [14]:
def plot_modal(df, cor, txt):
    for r in df['relacao'].values:
        p0 = conn.loc[ r[0] ]
        p1 = conn.loc[ r[1] ]
        plt.plot([p0['xcoord'], p1['xcoord']], [p0['ycoord'], p1['ycoord']],
                  linewidth=1, color=cor, alpha=0.6)
    # manipulador de legenda
    return mlines.Line2D([], [], color=cor, label=txt)


# based on:
# https://stackoverflow.com/questions/11551049/matplotlib-plot-zooming-with-scroll-wheel
class CanvasEventHandler:
    
    def __init__(self, ax):
        self.ax = ax
        self.zoom_count = 0
        self.base_scale = 2.0
        self.press_coord = None
        
        fig = ax.get_figure()
        fig.canvas.mpl_connect('scroll_event', self.on_scroll)
        fig.canvas.mpl_connect('button_press_event', self.on_press)
        fig.canvas.mpl_connect('button_release_event', self.on_release)
        fig.canvas.mpl_connect('motion_notify_event', self.on_motion)
    
    def refresh(self):
        fig = ax.get_figure()
        fig.canvas.draw()
        
    def delimit(self):
        self.cur_xlim = ax.get_xlim()
        self.cur_ylim = ax.get_ylim()
    
    def on_scroll(self, event):
        if self.press_coord is not None:
            return
        if event.button == 'up':
            if self.zoom_count == 3:
                return
            else:
                self.zoom_count += 1
                scale_factor = 1 / self.base_scale
        elif event.button == 'down':
            if self.zoom_count == 0:
                return
            else:
                self.zoom_count -= 1
                scale_factor = self.base_scale
        # get the current x and y limits
        self.delimit()
        xrange = (self.cur_xlim[1] - self.cur_xlim[0])*.5
        yrange = (self.cur_ylim[1] - self.cur_ylim[0])*.5
        xdata = event.xdata # get event x location
        ydata = event.ydata # get event y location
        # set new limits
        self.ax.set_xlim([xdata - xrange * scale_factor,
                          xdata + xrange * scale_factor])
        self.ax.set_ylim([ydata - yrange * scale_factor,
                          ydata + yrange * scale_factor])
        self.refresh()
        
    def on_press(self, event):
        if event.inaxes != self.ax:
            return
        self.delimit()
        self.press_coord = event.xdata, event.ydata

    def on_release(self, event):
        self.press_coord = None
        self.refresh()

    def on_motion(self, event):
        if (self.press_coord is None) or (event.inaxes != self.ax):
            return
        dx = event.xdata - self.press_coord[0]
        dy = event.ydata - self.press_coord[1]
        self.cur_xlim -= dx
        self.cur_ylim -= dy
        ax.set_xlim(self.cur_xlim)
        ax.set_ylim(self.cur_ylim)
        self.refresh()

In [15]:
# exibição do grafo interativo e exportação
#---------------------------------------------------------------------------------
#%matplotlib qt
#---------------------------------------------------------------------------------
fig, ax = plt.subplots(1,1,figsize=(15,15))
plt.rcParams.update({'font.size': 20})
plt.scatter(conn['xcoord'].values, conn['ycoord'].values, color='green', alpha=0.4)
l1 = plot_modal(df_r, 'purple', 'rodovias')
l2 = plot_modal(df_h, 'blue', 'hidrovias')
l3 = plot_modal(df_f, 'red', 'ferrovias')
#figuras em [pt]
plt.ylabel('Latitude (graus)')
plt.xlabel('Longitude (graus)')
plt.legend(handles=[l1, l2, l3])
plt.tight_layout()
plt.savefig(IMG + 'grafo-p1_[pt].png', dpi=100)
#figuras em [en]
plt.ylabel('Latitude (deg)')
plt.xlabel('Longitude (deg)')
plt.legend(handles=[l1, l2, l3], labels=['roadways', 'waterways', 'railways'])
plt.tight_layout()
plt.savefig(IMG + 'grafo-p1_[en].png', dpi=100);
#---------------------------------------------------------------------------------
plt.close()

#handler = CanvasEventHandler(ax)

In [16]:
# exportação dos grafos (sem exibição)
#---------------------------------------------------------------------------------
%matplotlib inline
#---------------------------------------------------------------------------------
plt.figure(figsize=(15,15))
plt.rcParams.update({'font.size': 20})
plt.scatter(conn['xcoord'].values, conn['ycoord'].values, color='green', alpha=0.4)
l4 = plot_modal(df_a, 'orange', 'rotas aéreas')
#figuras em [pt]
plt.ylabel('Latitude (graus)')
plt.xlabel('Longitude (graus)')
plt.legend(handles=[l4])
plt.tight_layout()
plt.savefig(IMG + 'grafo-p2_[pt].png', dpi=100)
#figuras em [en]
plt.ylabel('Latitude (deg)')
plt.xlabel('Longitude (deg)')
plt.legend(handles=[l4], labels=['air routes'])
plt.tight_layout()
plt.savefig(IMG + 'grafo-p2_[en].png', dpi=100)
#---------------------------------------------------------------------------------
plt.close()

In [17]:
plt.figure(figsize=(15,15))
plt.rcParams.update({'font.size': 20})
plt.scatter(conn['xcoord'].values, conn['ycoord'].values, color='green', alpha=0.4)
l5 = plot_modal(df_v, 'brown', 'fronteiras compartilhadas')
#figuras em [pt]
plt.ylabel('Latitude (graus)')
plt.xlabel('Longitude (graus)')
plt.legend(handles=[l5])
plt.tight_layout()
plt.savefig(IMG + 'grafo-p3_[pt].png', dpi=100)
#figuras em [en]
plt.ylabel('Latitude (deg)')
plt.xlabel('Longitude (deg)')
plt.legend(handles=[l5], labels=['shared borders'])
plt.tight_layout()
plt.savefig(IMG + 'grafo-p3_[en].png', dpi=100)
#---------------------------------------------------------------------------------
plt.close()